In [1]:
import requests

endpoint = 'http://localhost:9200'
index_name = 'boook'
index_mapping = {
    "mappings": {
        "properties": {
            "title": {"type": "text"},
            "url": {"type": "keyword"},
            "contributors": {"type": "keyword"},
            "date": {"type": "keyword"},
            "format": {"type": "keyword"},
            "full_text_url": {"type": "keyword"},
            "trove_id": {"type": "keyword"},
            "language": {"type": "keyword"},
            "rights": {"type": "keyword"},
            "pages": {"type": "integer"},
            "form": {"type": "keyword"},
            "volume": {"type": "keyword"},
            "children": {"type": "keyword"},
            "parent": {"type": "keyword"},
            "text_downloaded": {"type": "keyword"},
            "text_file": {"type": "keyword"}
        }
    }
}

response = requests.put(f'{endpoint}/{index_name}', json=index_mapping)
if response.status_code == 200:
    print("Index created successfully!")
else:
    print("Failed to create the index.")

Index created successfully!


In [1]:
from elasticsearch import Elasticsearch
import csv

es = Elasticsearch(hosts=['http://localhost:9200'])
index_name = 'boook'
file_path = 'books_info.csv'

def counter():
    counter.count += 1
    return counter.count
counter.count = 0

with open(file_path, 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    my_body = []
    for i, row in enumerate(reader, start=1):
        my_body.append({"index": {"_index": index_name, "_id": i}})
        my_body.append(dict(row))

print(len(my_body))
print(my_body[0])
print(my_body[1])
print(my_body[2])
print(my_body[3])

response = es.bulk(body=my_body)


84348
{'index': {'_index': 'boook', '_id': 1}}
{'title': 'Goliath Joe, fisherman / by Charles Thackeray (Wobbegong)', 'url': 'https://trove.nla.gov.au/work/10013347', 'contributors': 'Thackeray, Charles', 'date': '1900-1919', 'format': 'Book|Book/Illustrated', 'fulltext_url': 'https://nla.gov.au/nla.obj-2831231419', 'trove_id': 'nla.obj-2831231419', 'language': 'English', 'rights': 'Out of Copyright|http://rightsstatements.org/vocab/NKC/1.0/', 'pages': '130', 'form': 'Book', 'volume': '', 'parent': '', 'children': '', 'text_downloaded': 'True', 'text_file': 'goliath-joe-fisherman-by-charles-thackeray-wob-nla.obj-2831231419.txt'}
{'index': {'_index': 'boook', '_id': 2}}
{'title': 'Grammar of the Narrinyeri tribe of Australian Aborigines / by the late Rev. G. Taplin', 'url': 'https://trove.nla.gov.au/work/10029401', 'contributors': 'Taplin, George', 'date': '1878-1880', 'format': 'Book|Government publication', 'fulltext_url': 'http://nla.gov.au/nla.obj-688657424', 'trove_id': 'nla.obj-68

/tmp/ipykernel_5355/1609705154.py:26: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'operations' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  response = es.bulk(body=my_body)
/tmp/ipykernel_5355/1609705154.py:26: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.bulk(body=my_body)


In [4]:

session.execute(create_table_query_children)


In [4]:
from elasticsearch import Elasticsearch
from cassandra.cluster import Cluster
from cassandra.query import BatchStatement

es = Elasticsearch(hosts=['http://localhost:9200'])
print("Elasticsearch is ready.")
cluster = Cluster(['localhost'])
session = cluster.connect()
print("Cassandra is ready.\n")

index_name = 'boook'
batch_size = 1000
sort_field = '_doc'
search_after = None

insert_query_for_children = "INSERT INTO ali.children (trove_id,title,url,contributors,date,format,pages) VALUES (?,?,?,?,?,?,?)"
create_table_query_children= "CREATE TABLE ali.children (\
    title text,\
    url text,\
    contributors text,\
    date text,\
    format text,\
    trove_id text PRIMARY KEY,\
    pages int\
);"
insert_statement = session.prepare(insert_query_for_children)
while True:
    res = es.search(
        index='boook',
        body={
            'query': {
                'bool': {
                    'must': [
                        {'exists': {'field': 'parent'}},
                        {'bool': {'must_not': [{'term': {'parent': ''}}]}}
                    ]
                }
            },
            'sort': [{sort_field: {'order': 'asc'}}],
            'size': batch_size,
            'search_after': search_after
        }
    )

    if res['hits']['hits']:
        last_hit = res['hits']['hits'][-1]
        search_after = last_hit['sort']
        
        for hit in res['hits']['hits']:
            doc_id = hit['_id']
            doc = hit['_source']
            row = (doc['trove_id'], doc['title'], doc['url'], doc['contributors'], doc['date'], doc['format'], int(doc['pages']))
            session.execute(insert_statement, row)
    else:
        break

print("Done.")

Elasticsearch is ready.
Cassandra is ready.



/tmp/ipykernel_5603/3288612868.py:34: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(
/tmp/ipykernel_5603/3288612868.py:34: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = es.search(


Done.
Done.


In [6]:
from elasticsearch import Elasticsearch
from cassandra.cluster import Cluster
from cassandra.query import BatchStatement

es = Elasticsearch(hosts=['http://localhost:9200'])
print("Elasticsearch is ready.")
cluster = Cluster(['localhost'])
session = cluster.connect()
print("Cassandra is ready.\n")

index_name = 'boook'
batch_size = 1000
sort_field = '_doc'
search_after = None

insert_query_for_parents = "INSERT INTO ali.parents (trove_id,title,url,contributors,date,format,pages) VALUES (?,?,?,?,?,?,?)"
create_table_query_parents= "CREATE TABLE ali.parents (\
    title text,\
    url text,\
    contributors text,\
    date text,\
    format text,\
    trove_id text PRIMARY KEY,\
    pages int\
);"

insert_statement = session.prepare(insert_query_for_parents)

while True:
    res = es.search(
        index='boook',
        body={
            'query': {
                'bool': {
                    'must': [
                        {'exists': {'field': 'children'}},
                        {'bool': {'must_not': [{'term': {'children': ''}}]}}
                    ]
                }
            },
            'sort': [{sort_field: {'order': 'asc'}}],
            'size': batch_size,
            'search_after': search_after
        }
    )

    if res['hits']['hits']:
        last_hit = res['hits']['hits'][-1]
        search_after = last_hit['sort']
        
        for hit in res['hits']['hits']:
            doc_id = hit['_id']
            doc = hit['_source']
            row = (doc['trove_id'], doc['title'], doc['url'], doc['contributors'], doc['date'], doc['format'], int(doc['pages']))
            session.execute(insert_statement, row)
    else:
        break

print("Done.")

Elasticsearch is ready.
Cassandra is ready.

Done.


/tmp/ipykernel_5603/3932230067.py:35: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(
/tmp/ipykernel_5603/3932230067.py:35: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = es.search(
